# Table of Contents
  Example 2. Download huge ImageCollection from google earth engine; -> Download Images or ImageCollection
  Example 3. Extract NDVI values of climate stations from 1982 to 2021; -> Spatial analysis
  Example 4. Supervised Classification -> Images Classification
  Example 5. Unsupervised Classification (clustering) 

In [1]:
import ee, os
import geemap
# ee.Initialize()
Map = geemap.Map()
Map = geemap.Map(center=(39, 110), zoom=4)
Map

Map(center=[39, 110], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=HBox(children=(To…

In [63]:
# -------------------------------------------------------
# 2  ImageCollection 去云
#  Calculate NDVI for Jiangxi
# -------------------------------------------------------
year = 2021, 2021
doy = '-01-01', '-12-30'

start_date = str(year[0]) + doy[0]
end_date = str(year[1]) + doy[1]

jx_bound = ee.FeatureCollection('users/yehuigeo/jx_bound')
roi = jx_bound.geometry()

In [3]:
def cloudMaskL457(image):
    qa = image.select('pixel_qa')
  # If the cloud bit (5) is set and the cloud confidence (7) is high
  # or the cloud shadow bit is set (3), then it's a bad pixel.
    cloud = qa.bitwiseAnd(1 << 5) \
                  .And(qa.bitwiseAnd(1 << 7)) \
                  .Or(qa.bitwiseAnd(1 << 3))
  # Remove edge pixels that don't occur in all bands
    mask2 = image.mask().reduce(ee.Reducer.min())
    return image.updateMask(cloud.Not()).updateMask(mask2).divide(10000).copyProperties(image, ["system:time_start",'system:id'])

def maskS2clouds(image):   # This function was used to mask the clouds for sentinel-2
    qa = image.select('QA60')

  # Bits 10 and 11 are clouds and cirrus, respectively.
    cloudBitMask = 1 << 10
    cirrusBitMask = 1 << 11

  # Both flags should be set to zero, indicating clear conditions.
    mask = qa.bitwiseAnd(cloudBitMask).eq(0).And(
             qa.bitwiseAnd(cirrusBitMask).eq(0))

  # Return the masked and scaled data, without the QA bands.
    return image.updateMask(mask).divide(10000).select("B.*").copyProperties(image, ["system:time_start",'system:id'])

def maskL8sr(image):
  # Bits 3 and 5 are cloud shadow and cloud, respectively.
    cloudShadowBitMask = (1 << 3)
    cloudsBitMask = (1 << 5)
    # Get the pixel QA band.
    qa = image.select('pixel_qa')
    # Both flags should be set to zero, indicating clear conditions.
    mask = qa.bitwiseAnd(cloudShadowBitMask).eq(0) \
                 .And(qa.bitwiseAnd(cloudsBitMask).eq(0)).divide(10000)
    return image.updateMask(mask).copyProperties(image, ["system:time_start",'system:id'])

In [65]:
collection8 = ee.ImageCollection('LANDSAT/LC08/C01/T1_SR')\
    .filterBounds(jx_bound) \
    .filterDate(start_date, end_date).filter(ee.Filter.lt('CLOUD_COVER',20)) \
    .map(maskL8sr).select(['B2', 'B3', 'B4', 'B5', 'B6', 'B7', 'pixel_qa'],['B1', 'B2', 'B3', 'B4', 'B5', 'B7', 'pixel_qa']) 
collection8.size().getInfo()

96

In [66]:
# col8 = collection8
col8_ids = collection8.aggregate_array('system:id').getInfo()
# print(col8_ids)

In [67]:
f_dir = r'C:\Users\xiong\Downloads'
def dow(i):  # 用地图id做循环
    Layer = ee.Image(i).select(['B3', 'B2', 'B1']) # 使用地图，select为确定地图波段，可以去除，则为全波段
    # 保存文件名为地图id加文件格式
    a = i.split('/')[2:]
    b = '_'.join(a)
    f_img = f_dir + os.sep + b + '.tif'
    # 导出栅格影像，file_per_band为是否按波段分别导出
    geemap.ee_export_image(Layer, filename=f_img, scale=500, region=roi,file_per_band=False)
#     ee_export(region=roi, image=Layer.clip(roi), outfile=f_img, scale=1000, sep=0.25, num_workers=8)

In [68]:
from concurrent.futures.thread import ThreadPoolExecutor

with ThreadPoolExecutor(max_workers=8) as executor:
    executor.map(dow, col8_ids)

Generating URL ...
Generating URL ...
Generating URL ...
Generating URL ...
Generating URL ...
Generating URL ...
Generating URL ...
Generating URL ...
Please wait ...
Please wait ...
Please wait ...
Please wait ...
Please wait ...
Please wait ...
Please wait ...
Please wait ...
Data downloaded to C:\Users\YeHui\Downloads\C01_T1_SR_LC08_120039_20210326.tif
Generating URL ...
Data downloaded to C:\Users\YeHui\Downloads\C01_T1_SR_LC08_120039_20211223.tif
Generating URL ...
Data downloaded to C:\Users\YeHui\Downloads\C01_T1_SR_LC08_120039_20210918.tif
Generating URL ...
Data downloaded to C:\Users\YeHui\Downloads\C01_T1_SR_LC08_120039_20211004.tif
Generating URL ...
Data downloaded to C:\Users\YeHui\Downloads\C01_T1_SR_LC08_120039_20211207.tif
Generating URL ...
Please wait ...
Data downloaded to C:\Users\YeHui\Downloads\C01_T1_SR_LC08_120040_20210326.tif
Generating URL ...
Data downloaded to C:\Users\YeHui\Downloads\C01_T1_SR_LC08_120040_20210902.tif
Generating URL ...
Data downloaded to

Please wait ...
Data downloaded to C:\Users\YeHui\Downloads\C01_T1_SR_LC08_121039_20211128.tif
Generating URL ...
Please wait ...
Data downloaded to C:\Users\YeHui\Downloads\C01_T1_SR_LC08_121040_20210112.tif
Generating URL ...
Data downloaded to C:\Users\YeHui\Downloads\C01_T1_SR_LC08_121040_20210808.tif
Generating URL ...
Data downloaded to C:\Users\YeHui\Downloads\C01_T1_SR_LC08_121040_20210605.tif
Generating URL ...
Data downloaded to C:\Users\YeHui\Downloads\C01_T1_SR_LC08_121041_20210112.tif
Generating URL ...
Please wait ...
Data downloaded to C:\Users\YeHui\Downloads\C01_T1_SR_LC08_121040_20211112.tif
Generating URL ...
Data downloaded to C:\Users\YeHui\Downloads\C01_T1_SR_LC08_121041_20210925.tif
Generating URL ...
Data downloaded to C:\Users\YeHui\Downloads\C01_T1_SR_LC08_121040_20210925.tif
Generating URL ...
Data downloaded to C:\Users\YeHui\Downloads\C01_T1_SR_LC08_121041_20211112.tif
Generating URL ...
Please wait ...
Please wait ...
Please wait ...
Please wait ...
Please

Please wait ...
Please wait ...
Data downloaded to C:\Users\YeHui\Downloads\C01_T1_SR_LC08_122039_20211103.tif
Generating URL ...
Data downloaded to C:\Users\YeHui\Downloads\C01_T1_SR_LC08_122040_20210119.tif
Generating URL ...
Data downloaded to C:\Users\YeHui\Downloads\C01_T1_SR_LC08_122040_20210220.tif
Generating URL ...
Data downloaded to C:\Users\YeHui\Downloads\C01_T1_SR_LC08_122040_20210324.tif
Generating URL ...
Please wait ...
Data downloaded to C:\Users\YeHui\Downloads\C01_T1_SR_LC08_122040_20210714.tif
Generating URL ...
Please wait ...
Data downloaded to C:\Users\YeHui\Downloads\C01_T1_SR_LC08_122040_20210831.tif
Generating URL ...
Please wait ...
Please wait ...
Please wait ...
Data downloaded to C:\Users\YeHui\Downloads\C01_T1_SR_LC08_122040_20210916.tif
Generating URL ...
Please wait ...
Please wait ...
Data downloaded to C:\Users\YeHui\Downloads\C01_T1_SR_LC08_122040_20211002.tif
Generating URL ...
Data downloaded to C:\Users\YeHui\Downloads\C01_T1_SR_LC08_122040_202112

Data downloaded to C:\Users\YeHui\Downloads\C01_T1_SR_LC08_122043_20210220.tif
Generating URL ...
Please wait ...
Data downloaded to C:\Users\YeHui\Downloads\C01_T1_SR_LC08_122043_20211205.tif
Generating URL ...
Please wait ...
Data downloaded to C:\Users\YeHui\Downloads\C01_T1_SR_LC08_122043_20211002.tif
Generating URL ...
Please wait ...
Data downloaded to C:\Users\YeHui\Downloads\C01_T1_SR_LC08_123039_20210923.tif
Generating URL ...
Please wait ...
Please wait ...
Data downloaded to C:\Users\YeHui\Downloads\C01_T1_SR_LC08_123039_20211110.tif
Generating URL ...
Data downloaded to C:\Users\YeHui\Downloads\C01_T1_SR_LC08_123040_20210923.tif
Generating URL ...
Please wait ...
Data downloaded to C:\Users\YeHui\Downloads\C01_T1_SR_LC08_123040_20211025.tif
Generating URL ...
Please wait ...
Data downloaded to C:\Users\YeHui\Downloads\C01_T1_SR_LC08_123040_20211126.tif
Please wait ...
Please wait ...
Data downloaded to C:\Users\YeHui\Downloads\C01_T1_SR_LC08_123041_20210822.tif
Data downloa

In [11]:
sl2 = sentinel2.select(['mNDWI']).toBands()
# merged = sl2.iterate(merge_bd, sl2.first())

In [ ]:
from GEE import ee_export
f_img = r"C:\Users\xiong\Downloads\sl2_mNDWI"
roi = jx_bound.geometry()
ee_export(region=roi, image=sl2, outfile=f_img, scale=10, sep=0.25, num_workers=8)

In [18]:
image = sentinel2.select('mNDWI').median()
Msize = image.get('system:asset_size').getInfo()
print(Msize)

None


In [36]:
name = sentinel2.select('mNDWI').aggregate_array('system:id').getInfo()
print(name)

['COPERNICUS/S2_SR/20210605T024549_20210605T025350_T49RHJ', 'COPERNICUS/S2_SR/20210605T024549_20210605T025350_T50RKP', 'COPERNICUS/S2_SR/20210605T024549_20210605T025350_T50RLR', 'COPERNICUS/S2_SR/20210605T024549_20210605T025350_T50RLS', 'COPERNICUS/S2_SR/20210605T024549_20210605T025350_T50RLT', 'COPERNICUS/S2_SR/20210605T024549_20210605T025350_T50RMR', 'COPERNICUS/S2_SR/20210605T024549_20210605T025350_T50RMS', 'COPERNICUS/S2_SR/20210605T024549_20210605T025350_T50RMT', 'COPERNICUS/S2_SR/20210605T024549_20210605T025350_T50RMU', 'COPERNICUS/S2_SR/20210605T024549_20210605T025350_T50RNS', 'COPERNICUS/S2_SR/20210605T024549_20210605T025350_T50RNT', 'COPERNICUS/S2_SR/20210605T024549_20210605T025350_T50RNU', 'COPERNICUS/S2_SR/20210605T024549_20210605T025350_T50RPS', 'COPERNICUS/S2_SR/20210605T024549_20210605T025350_T50RPT', 'COPERNICUS/S2_SR/20210608T025549_20210608T030747_T50RLU', 'COPERNICUS/S2_SR/20210608T025549_20210608T031000_T49RGL', 'COPERNICUS/S2_SR/20210608T025549_20210608T031000_T50RK

In [ ]:
dow_img_dir = r'C:\Users\xiong\Downloads'
from tolocal import ee_export
roi = jx_bound.geometry()
def dow(i):  # 用地图id做循环
    Layer = ee.Image(i).select(['B4', 'B3', 'B2']) # 使用地图，select为确定地图波段，可以去除，则为全波段
    # 保存文件名为地图id加文件格式
    a = i.split('/')
    str = '-'
    b = str.join(a)
    file = dow_img_dir + os.sep + b
    # 导出栅格影像，file_per_band为是否按波段分别导出
#     ee_export(region=roi, image=Layer.clip(roi), outfile=file, scale=10, sep=0.1, num_workers=4)
    geemap.ee_export_image(Layer, filename=file, scale=500, region=roi,file_per_band=True)

In [ ]:
from concurrent.futures.thread import ThreadPoolExecutor

with ThreadPoolExecutor(max_workers=8) as executor:
    executor.map(dow, names_id)

In [ ]:
roi = jx_bound.geometry()
Layer = ee.Image('COPERNICUS/S2_SR/20210601T000729_20210601T000731')

In [ ]:
for i in names_id:
    i = r"COPERNICUS/S2_SR/20210605T024549_20210605T025350_T49RHJ"
    Layer = ee.Image(i).select(['B4', 'B3', 'B2']) # 使用地图，select为确定地图波段，可以去除，则为全波段
    # 保存文件名为地图id加文件格式
    a = i.split('/')
    str = '-'
    b = str.join(a)
    file = dow_img_dir + os.sep + b
    ee_export(region=roi, image=Layer.clip(roi), outfile=file, scale=10, sep=0.1, num_workers=4)

In [ ]:
from tolocal import ee_export
ee_export(region=roi, image=Layer.clip(roi), outfile=file, scale=10, sep=0.1, num_workers=4)

In [ ]:
img = sentinel2.select(['B4', 'B3', 'B2']).median().clip(roi)
print(img.propertyNames().getInfo())

In [ ]:
Map.addLayer(sentinel2.select(['B4', 'B3', 'B2']).median().clip(roi),{'bands': ['B4', 'B3', 'B2'], 'min': 0, 'max': 0.3},"Layer")

In [ ]:
a = sl1.toList(100).getInfo()

In [ ]:
a = ee.ImageCollection("COPERNICUS/S2_SR").select('mNDWI').filterDate(start_date, end_date).filterBounds(roi)

In [ ]:
a.size().getInfo()